In [9]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from IPython.display import display
import os
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
print(f"Parent directory: {parent_dir}")

Parent directory: /u/weim/lob


In [ ]:
lots_list = [20,60]
n_samples = 10000
train_iter = 400

folder_path = f"{parent_dir}/rewards"
data = {}
# agent_list = ['sl_agent', 'linear_sl_agent', 'log_normal', 'dirichlet']
agent_list = ['sl_agent', 'linear_sl_agent', 'log_normal', 'log_normal_raw']
for env in ['noise', 'flow', 'strategic']:
    data[env] = {}
    for lots in lots_list:
        data[env][lots] = {}
        for agent in agent_list:            
            try: 
                if agent == 'log_normal':    
                    name = f'{folder_path}/{env}_{lots}_seed_0_eval_seed_100_eval_episodes_{n_samples}_num_iterations_{train_iter}_bsize_12800_{agent}.npz'           
                    name='/u/weim/lob/rewards/noise_20_seed_0_eval_seed_100_eval_episodes_10000_num_iterations_400_bsize_12800_log_normal.npz'
                elif agent == 'log_normal_raw':
                    name =  f'{folder_path}/{env}_{lots}_seed_0_eval_seed_100_eval_episodes_10000_num_iterations_{train_iter}_bsize_12800_{agent}_rewards.npz'
                else:
                    name = f'{folder_path}/{env}_{lots}_episodes_{10000}_eval_seed_100_{agent}.npz'
                print(f'loading from {name}')
                data[env][lots][agent] = np.load(name)['rewards']
                if agent in ['log_normal_raw']:
                    # normalize rewards 
                    initial_bid = 1000
                    data[env][lots][agent] = data[env][lots][agent]/lots - initial_bid
            except:
                print(f"path doesnt exist: {name}")
                data[env][lots][agent] = 'no data aviailable'

print(data['noise'][20]['log_normal_raw'])

loading from /u/weim/lob/rewards/noise_20_episodes_10000_eval_seed_100_sl_agent.npz
loading from /u/weim/lob/rewards/noise_20_episodes_10000_eval_seed_100_linear_sl_agent.npz
loading from /u/weim/lob/rewards/noise_20_episodes_10000_eval_seed_100_log_normal.npz
path doesnt exist: /u/weim/lob/rewards/noise_20_episodes_10000_eval_seed_100_log_normal.npz
loading from /u/weim/lob/rewards/noise_20_seed_0_eval_seed_100_eval_episodes_10000_num_iterations_400_bsize_12800_log_normal_raw_rewards.npz
loading from /u/weim/lob/rewards/noise_60_episodes_10000_eval_seed_100_sl_agent.npz
loading from /u/weim/lob/rewards/noise_60_episodes_10000_eval_seed_100_linear_sl_agent.npz
loading from /u/weim/lob/rewards/noise_60_episodes_10000_eval_seed_100_log_normal.npz
path doesnt exist: /u/weim/lob/rewards/noise_60_episodes_10000_eval_seed_100_log_normal.npz
loading from /u/weim/lob/rewards/noise_60_seed_0_eval_seed_100_eval_episodes_10000_num_iterations_400_bsize_12800_log_normal_raw_rewards.npz
loading from

In [3]:
folder_path = f"{parent_dir}/latex_tables"
list_of_dfs = []
# n_lots = [20, 60]
# print(filler)
for env in ['noise', 'flow', 'strategic']:
    # filler = pd.DataFrame(np.nan*np.ones((1,6)), columns=['E[SL]', 'Std[SL]', 'E[TWAP]', 'Std[TWAP]', 'E[RL]', 'Std[RL]'], index=[f'{env}'])
    filler = pd.DataFrame(np.nan*np.ones((1,8)), columns=['E[SL]', 'Std[SL]', 'E[TWAP]', 'Std[TWAP]', 'E[LN]', 'Std[LN]', 'E[LNR]', 'Std[LNR]'], index=[f'{env}'])
    list_of_dfs.append(filler)
    # print(f'{env}: ')
    data_for_df = {}
    # name_dict = {'sl_agent': 'SL', 'linear_sl_agent': 'TWAP', 'actor_critic': 'RL'}
    name_dict = {'sl_agent': 'SL', 'linear_sl_agent': 'TWAP', 'log_normal': 'LN', 'log_normal_raw': 'LNR'}
    agent_names = ['sl_agent', 'linear_sl_agent', 'log_normal', 'log_normal_raw']
    for agent in agent_names:
        name = name_dict[agent]
        data_for_df[f'E[{name}]'] = []
        data_for_df[f'Std[{name}]'] = []
        for lots in lots_list:
            try:
                data_for_df[f'Std[{name}]'] += [np.std(data[env][lots][agent])]
                data_for_df[f'E[{name}]'] += [np.mean(data[env][lots][agent])]                
            except:
                print(f"no data available for {env}, {lots}, {agent}")
                data_for_df[f'Std[{name}]'] += [np.nan]
                data_for_df[f'E[{name}]'] += [np.nan]
    df = pd.DataFrame.from_dict(data_for_df).round(2)    
    df.index.name = 'Lots'
    index_name = [f'{env}_{l}' for l in lots_list]
    df.index = index_name
    df = df.round(2)
    list_of_dfs.append(df)

complete_df = pd.concat(list_of_dfs, axis=0)
print(complete_df)
latex_table = complete_df.to_latex(float_format="%.2f")
with open(f"{folder_path}/results_table_latest_with_raw.tex", "w") as f:
    f.write(latex_table)

no data available for noise, 20, log_normal_raw
no data available for noise, 60, log_normal_raw
no data available for flow, 20, log_normal_raw
no data available for flow, 60, log_normal_raw
no data available for strategic, 20, log_normal_raw
no data available for strategic, 60, log_normal_raw
              E[SL]  Std[SL]  E[TWAP]  Std[TWAP]  E[LN]  Std[LN]  E[LNR]  \
noise           NaN      NaN      NaN        NaN    NaN      NaN     NaN   
noise_20       0.52     1.19    -0.06       0.94  -0.61     0.53     NaN   
noise_60      -1.09     1.34    -1.40       0.98  -1.17     0.82     NaN   
flow            NaN      NaN      NaN        NaN    NaN      NaN     NaN   
flow_20        0.10     1.43     0.48       0.68  -0.29     0.39     NaN   
flow_60       -3.36     0.99    -0.96       0.95  -1.02     0.45     NaN   
strategic       NaN      NaN      NaN        NaN    NaN      NaN     NaN   
strategic_20  -1.64     2.95    -0.36       3.03  -0.37     2.23     NaN   
strategic_60  -2.51   